In [ ]:
# Task 1: File Management Script
'''
Write a Bash script that
- Creates a directory named "backup" in the user's home directory
- Copies all .txt files from the current directory into the "backup" directory
- Appends the current date and time to the filenames of the copied files. '''

In [ ]:
Task 1 - File Management Script - Solution

#!/bin/bash

BACKUP_DIR="$HOME/backup"

mkdir -p "$BACKUP_DIR"

TIMESTAMP=$(date +"%Y-%m-%d_%H:%M:%S")

for file in *.txt; do
        [ -e "$file" ] || { echo "No .txt file exists."; exit 1; }
        filename=$(basename "$file" .txt)
        cp "$file" "$BACKUP_DIR/${filename}_${TIMESTAMP}.txt"
done

echo
echo "Backup completed successfully. Files are saved in $BACKUP_DIR"

In [ ]:
# Task 2: System Health Check
'''
Create a script that
- Checks the system’s CPU and memory usage
- Reports if the CPU usage is above 80% or if the available memory is below 20%
- Logs the results to a file named system_health.log. '''

In [ ]:
Task 2 - System Health Check - Solution

#!/bin/bash

LOG_FILE="$HOME/Desktop/bash_dir/system_health.log"
TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')

DISK_USAGE=$(df -H / | awk 'NR==2 {gsub("%","",$5); print int($5)}')
DISK_FREE=$((100 - DISK_USAGE))

CPU_IDLE=$(top -bn1 | awk '/Cpu\(s\)/ {print int($8)}')
CPU_USAGE=$((100 - CPU_IDLE))

if [[ "$DISK_FREE" -lt 20 || "$CPU_USAGE" -gt 80 ]]; then
        echo "$TIMESTAMP - System Warning! - Disk Available: ${DISK_FREE}%, CPU Usage: ${CPU_USAGE}%" >> $LOG_FILE
else
        echo "$TIMESTAMP - System Ok! - Disk Available: ${DISK_FREE}%, CPU Usage: ${CPU_USAGE}%" >> $LOG_FILE
fi

echo
echo "The system health check completed successfully."

In [ ]:
# Task 3: User Account Management
'''
Write a script that
- Reads a list of usernames from a file (e.g., user_list.txt)
- Creates a new user for each username
- Generates a random password for each user and saves the username and password to a file named
credentials.txt '''

In [ ]:
Task 3 - User Account Management - Solution

#!/bin/bash

USER_LIST="$HOME/user_list.txt"
CREDENTIALS_FILE="$HOME/credentials.txt"

if [[ ! -f "$USER_LIST" ]]; then
        echo "Error: $USER_LIST not found!"
        exit 1
else
        echo "$USER_LIST is found. Processing..."
fi

while IFS= read -r USERNAME; do
        [[ -z "$USERNAME" ]] && continue
        PASSWORD=$(openssl rand -base64 9)

        if id "$USERNAME" &>/dev/null; then
                echo "User $USERNAME already exists. Skipping..."
        else
                sudo useradd "$USERNAME"
                echo "$USERNAME:$PASSWORD" | sudo chpasswd
                echo "Created user $USERNAME"
        fi
        echo "$USERNAME : $PASSWORD" >> "$CREDENTIALS_FILE"
done < "$USER_LIST"

echo
echo "All users processed successfully."


In [ ]:
# Task 4: Automated Backup
'''
Create a script that
- Takes a directory path as input from the user
- Compresses the directory into a .tar.gz file
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz). '''

In [ ]:
Task 4 - Automated Backup - Solution

#!/bin/bash

read -p "Enter the directory path for backup: " DIR_PATH

if [ ! -d "$DIR_PATH" ]; then
        echo "ERROR: Directory '${DIR_PATH}' not found!"
        exit 1
else
        echo "SUCCESS: Directory '${DIR_PATH}' found! Processing..."
fi

TIMESTAMP=$(date +"%Y-%m-%d")
BACKUP_DIR="$HOME/Backup_dir"

mkdir -p "$BACKUP_DIR"

BACKUP_FILE="$BACKUP_DIR/backup_${TIMESTAMP}.tar.gz"

echo "Starting backup of '${DIR_PATH}'..."
sleep 1

tar -czf "$BACKUP_FILE" -C "$(dirname "$DIR_PATH")" "$(basename "$DIR_PATH")"

if [ $? -eq 0 ]; then
        echo "Backup successful! Archive created at: $BACKUP_FILE"
else
        echo "Backup failed. Check for errors."
        rm "$BACKUP_FILE"
        exit 1
fi

In [ ]:
# Task 5: Simple To-Do List
'''
Create a Bash script that
- Implements a simple command-line to-do list
- Allows the user to add tasks, view tasks, and remove tasks
- Saves the tasks to a file (e.g., todo.txt). '''

In [ ]:
Task 5 - Simple To-Do List - Solution

#!/bin/bash

TODO_FILE="$HOME/Desktop/bash_dir/todo.txt"

touch "$TODO_FILE"

show_menu() {
        echo
        echo "====== 📝 SIMPLE TO-DO LIST ======"
        echo "1️⃣  View Tasks"
        echo "2️⃣  Add Task"
        echo "3️⃣  Remove Task"
        echo "4️⃣  Exit"
        echo "=================================="
        echo
}

view_tasks() {
        if [[ ! -s "$TODO_FILE" ]]; then
                echo "No tasks found. Your to-do list is empty."
        else
                echo "Your current tasks:"
                nl -w2 -s'. ' "$TODO_FILE"
        fi
}

add_task() {
        read -p "Enter your new task: " TASK
        if [[ -z "$TASK" ]]; then
                echo "Task cannot be empty."
        else
                echo "$TASK" >> "$TODO_FILE"
                echo "Task added successfully!"
        fi
}

remove_task() {
        view_tasks
        if [[ ! -s "$TODO_FILE" ]]; then
                return
        fi
        read -p "Enter the task number to remove: " NUM
        if [[ "$NUM" =~ ^[0-9]+$ ]]; then
                sed -i "${NUM}d" "$TODO_FILE"
                echo "Task #$NUM removed successfully!"
        else
                echo "Invalid input. Please enter a number."
        fi
}

while true; do
        show_menu
        read -p "Choose an option (1-4): " CHOICE
        case "$CHOICE" in
                1) view_tasks ;;
                2) add_task ;;
                3) remove_task ;;
                4) echo "Goodbye!"; exit 0 ;;
                *) echo "Invalid choice. Please try again." ;;
        esac
done

In [ ]:
# Task 6: Automated Software Installation
'''
Write a script that
- Reads a list of software package names from a file (e.g., packages.txt)
- Installs each package using the appropriate package manager (apt, yum, etc.)
- Logs the installation status of each package. '''

In [ ]:
Task 6 - Automated Software Installation - Solution

#!/bin/bash

PKG_FILE="packages.txt"

LOG_FILE="$HOME/Desktop/bash_dir/package_install.log"

if [[ ! -f "$PKG_FILE" ]]; then
        echo "ERROR: File '$PKG_FILE' not found!"
        exit 1
fi

if command -v apt &>/dev/null; then
        PKG_MANAGER="apt"
        INSTALL_CMD="sudo apt install -y"
elif command -v dnf &>/dev/null; then
        PKG_MANAGER="dnf"
        INSTALL_CMD="sudo dnf install -y"
elif command -v yum &>/dev/null; then
        PKG_MANAGER="yum"
        INSTALL_CMD="sudo yum install -y"
elif command -v pacman &>/dev/null; then
        PKG_MANAGER="pacman"
        INSTALL_CMD="sudo pacman -S --noconfirm"
else
        echo "ERROR: No supported package manager found (apt, dnf, yum, pacman)." | tee -a "$LOG_FILE"
        exit 1
fi

echo "Using package manager: $PKG_MANAGER"
echo "Logging installation results to: $LOG_FILE"
echo "------------------------------------------" >> "$LOG_FILE"
echo "Installation Log - $(date)" >> "$LOG_FILE"
echo "------------------------------------------" >> "$LOG_FILE"

while IFS= read -r PACKAGE || [[ -n "$PACKAGE" ]]; do
        [[ -z "$PACKAGE" ]] && continue
        echo "Installing: $PACKAGE ..."
        if $INSTALL_CMD "$PACKAGE" &>/dev/null; then
                echo "SUCCESS: $PACKAGE installed successfully." | tee -a "$LOG_FILE"
        else
                echo "FAILED: $PACKAGE installation failed." | tee -a "$LOG_FILE"
        fi
done < "$PKG_FILE"

echo
echo "Installation process complete. Check '$LOG_FILE' for details."


In [ ]:
# Task 7: Text File Processing
'''
Create a script that
- Takes a text file as input
- Counts and displays the number of lines, words, and characters in the file
- Finds and displays the longest word in the file.'''

In [ ]:
Task 7 - Text File Processing - Solution

#!/bin/bash

read -p "Enter the file(with path): " FILE_PATH

if [ ! -f "$FILE_PATH" ]; then
        echo "ERROR: File '${FILE_PATH}' not found!"
        exit 1
else
        echo "SUCCESS: File '${FILE_PATH}' found! Processing..."
fi

LINES=$(wc -l < "$FILE_PATH")
WORDS=$(wc -w < "$FILE_PATH")
CHARS=$(wc -m < "$FILE_PATH")

LONGEST_WORD=$(tr -s '[:space:]' '\n' < "$FILE_PATH" | awk '{ print length, $0 }' | sort -nr | head -n 1 | awk '{ print $2 }')

echo "Number of lines: $LINES"
echo "Number of words: $WORDS"
echo "Number of chars: $CHARS"
echo "Longest word is: $LONGEST_WORD"